In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON, POST
import json

categories_all=None
with open('categories.json') as categories_file:
    categories_all = json.load(categories_file)

path_to_annotations='coco_anno/scenario1.json'
path_to_server = '/data/nginx/fileServer'

#Query Data from Knowledge Graph
query1_sparql = open("query1.sparql", "r").read()

sparql = SPARQLWrapper("http://172.17.0.10:8080/bigdata/sparql")                      
sparql.setReturnFormat(JSON)
sparql.setMethod(POST)
sparql.setQuery(query1_sparql)
results = sparql.query().convert()

In [2]:
#Query information of an annotation
def queryAnno(annoURI, annId, imageId):
    queryAnno = open('query/anno.sparql').read().replace('?annotation','<'+annoURI+'>')
    sparql.setQuery(queryAnno)
    result = sparql.query().convert()['results']['bindings'][0]
    x = float(result['x']['value'])
    y = float(result['y']['value'])
    w = float(result['w']['value'])
    h = float(result['h']['value'])
    area = w * h
    category_id = int(result['catId']['value'])
    bbox = [x,y,w,h]
    annotation = {
        'id':annId,
        'image_id':imageId,
        'bbox':bbox,
        'category_id':category_id,
        'area':area,
        'iscrowd': 0,
    }
    return annotation    

In [3]:
#Query information of an image
def queryImage(imageURI):
    queryImage = open('query/image.sparql').read().replace('?image','<'+imageURI+'>')
    sparql.setQuery(queryImage)
    result = sparql.query().convert()['results']['bindings'][0]
    
    id = int(imageURI.split('/')[-1])
    w  = int(result['w']['value'])
    h  = int(result['h']['value'])
    file_name = result['fileName']['value']
    path = result['path']['value']
    image = {
        'id':id,
        'file_name': file_name,
        'image_path': path_to_server + path,
        'width':w,
        'height':h
    }
    return image

In [ ]:
categories = []
images = []
annotations=[]
ids = []
annId=0

for result in results['results']['bindings']:
    imageId = int(result['image']['value'].split('/')[-1])
    imageURI = result['image']['value']
    if imageId not in ids:
        image=queryImage(imageURI)
        images.append(image)
        ids.append(imageId)
    
    annoURI = result['annotation']['value']
    annId+=1
    annotation = queryAnno(annoURI,annId, imageId)
    annotations.append(annotation)
    
dataset = {
    "images":images,
    "annotations":annotations,
    "categories" : categories_all
}        
 
with open(path_to_annotations,'w') as outfile:
    json.dump(dataset,outfile)

In [20]:
#Training code starts from here
path_to_annotations='coco_anno_examples/annotations/scenario1.json'

